# Bootstrap filter
Reference <br>
- https://www.cs.ubc.ca/~arnaud/doucet_johansen_tutorialPF.pdf
- https://link.springer.com/chapter/10.1007/978-1-4757-3437-9_1
- https://stats.stackexchange.com/questions/237468/bootstrap-filter-particle-filter-algorithmunderstanding

This technique is used to infere unobserved random variable $X$ using the observations $Y$. For example, the $X$ could be our temperature values and $Y$ could be some measurements of these temperatures using a sensore with some noise.

- **Transitions**:
$X_t/X_{t-1} \sim f(x_t/x_{t-1})$

- **Observations**:
$Y_t/X_{t} \sim g(y_t/x_{t})$

- **Initial state** :
$X_0  \sim \mu(x_0)$

Here, bootstrap filter is applied to the following example:

- Transitions:
 $X_t/X_{t-1} \sim N(x_{t-1},\sigma_f)$

- Observations:
$Y_t/X_{t} \sim N(y_t,\sigma_g)$

- Initial state:
$X_0  \sim N(\mu_0,\sigma_{\mu})$

In [26]:
# import
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patches as patches
import seaborn as sns
import scipy as sp
from scipy.stats import norm
from scipy import linalg
from scipy.stats import halfnorm
from ipywidgets import interactive
from IPython.display import clear_output
import timeit
#%matplotlib inline

In [27]:
#set figure defaults for IPython notebook 
#matplotlib.rcParams.update({'font.size': 18, 'lines.linewidth':4})

# A prelimanary example , 1-node

Infering the temperature at a location:
- given initial distirbution of temperature at that location
- Noisy measurements of temperature at that location
- Transition distirbution of tmeperature at a time given the previous time step temperature value

Inputs: <br>
- sensor measurements variance
- transition variance
- initial distibution mean and variance

Assumptions:
- Distirbutions are normal

Output:
- Infering mean and variance of state variable

**A visualization function**

for simple 1 node example

In [28]:
def plot_distribution(start_t,end_t,step_t,t_out,X_true,obs,Sample,pred):
    plt.figure(figsize=(10,5))
    plt.plot(list(np.arange(start_t,end_t+step_t,step_t)),X_true, color='red', linewidth = 2, label = 'True X')
    plt.scatter(list(np.arange(start_t,end_t+step_t,step_t)),obs, color='blue', label = 'Observations')
    plt.plot(list(np.arange(start_t,end_t+step_t,step_t)),pred[1:], color='green', linewidth = 2, label = 'Predictions')
    print('Average error between true X and observations is:', round(np.sum(abs(obs-X_true))/np.sum(abs(X_true))*100,2))
    print('Average error between true X and predicted values is:', round(np.sum(abs(pred[1:]-X_true))/np.sum(abs(X_true))*100,2))
    plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)
    
    plt.figure(figsize=(10,5))
    histogram = plt.hist(Sample[:,int((t_out-start_t)/step_t+1)], bins=int(N/100), label = "Distribution at time {}".format(t_out))
    x_true_t = X_true[int((t_out-start_t)/step_t+1)] # true value at time t_out
    obs_t = obs[int((t_out-start_t)/step_t+1)] # true value at time t_out
    pred_t = pred[int((t_out-start_t)/step_t+1)] # prediction value at time t_out
    plt.plot([x_true_t,x_true_t],[0,100], color='red', linewidth = 3, label = "True value at time {}".format(t_out))
    plt.plot([obs_t,obs_t],[0,100], color='black', linewidth = 3, label = "Observation at time {}".format(t_out))
    plt.plot([pred_t,pred_t],[0,100], color='green', linewidth = 3, label = "Distribution mean at time {}".format(t_out))
    plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)

for temperature distribution examples

In [43]:
def plot_temp_error_shade(t,t_start,t_end,delt,Coords,T_mean,T_var,air_temp_type,
                          T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma,T_ini,
                          sensor_loc_typ,sensor_loc_list,Length_c,Length_t,num_el_c,num_el_t):
    Length = Length_c + Length_t
    num_el = num_el_c + num_el_t
    # vizualizing mean and variacen of temperature for the rod    
    if sensor_loc_typ == "node":
        sensor_loc_n = sensor_loc_list # a list, node numbers
        sensor_loc = [min((i-1),num_el_t) * (Length_t/num_el_t) +
                      max(0, (i-1-num_el_t)) * (Length_c/num_el_c) for i in sensor_loc_n] 
    elif sensor_loc_typ == "loc":
        sensor_loc = sensor_loc_list # a list, location of sensor (m)
        #sensor_loc_n = [int(round(x /  (Length/num_el))) + 1 for x in sensor_loc] # sensor location node number
        sensor_loc_n = [int(min(num_el_t, round( x / (Length_t / num_el_t)) ) +
                            max(0, round( (x-Length_t) /  (Length_c/num_el_c))) ) + 1 for x in sensor_loc] # sensor location node number

    tn = int((t-t_start)/delt)
    plt.figure(figsize=(10,10))  
    plt.subplot(311)
    plt.fill_between(Coords, T_mean[:, tn]-T_var[:, tn], T_mean[:, tn]+T_var[:, tn], alpha=0.5)
    #plt.plot(Coords, T_mean[:, tn], color = 'k', linewidth=1)
    plt.plot(Coords[0:num_el_t+1], T_mean[0:num_el_t+1, tn], color = 'red', linewidth=1)
    plt.plot(Coords[num_el_t:num_el_t+num_el_c+2], T_mean[num_el_t:num_el_t+num_el_c+2, tn], color = 'k', linewidth=1)
    
    if len(sensor_loc)>0: # if there is any sensor
        plt.plot([sensor_loc,sensor_loc],[round(np.amin(T_mean)),round(np.amax(T_mean))*1.1]
                 , color = 'green', linewidth=1)
    plt.ylim([np.amin(T_mean)-np.amax(T_var),np.amax(T_mean)+np.amax(T_var)])
    if len(sensor_loc)>0: # if there is any sensor
        plt.title('Sensor location (shown as green line)= {} m \nSensor is at node(s)  {}.'
                  .format(sensor_loc, sensor_loc_n), fontsize=14, loc="left")
    plt.xlabel('location (m)')
    plt.ylabel('Temperature (K)')
    plt.show()
    # air temperature evolution
    plt.figure(figsize=(10,10))
    plt.subplot(312)
    T_air_arr = np.array([T_air(0,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma)])
    for time in np.arange(t_start+delt,t_end,delt):
        T_air_arr = np.append(T_air_arr, T_air(time,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma))         
    plt.plot(np.arange(t_start,t_end,delt),T_air_arr)
    plt.plot([t,t],[round(np.amax(T_air_arr)),round(np.amax(T_air_arr))*1.1], color = 'red', linewidth=1)
    plt.xlabel('Time (s)')
    plt.ylabel('Air temperature (K)')
    plt.title('Current time is shown as red line', fontsize=14, loc="left")
    plt.ylim(round(T_ini[0,0])-1,round(np.amax(T_air_arr))*1.1)
    plt.show()
    # std at different nodes 
    plt.figure(figsize=(10,10)) 
    plt.subplot(313) 
    #plt.plot(Coords, T_var[:, tn], color = 'k', linewidth=1)
    plt.plot(Coords[0:num_el_t+1], T_var[0:num_el_t+1, tn], color = 'red', linewidth=1)
    plt.plot(Coords[num_el_t:num_el_t+num_el_c+2], T_var[num_el_t:num_el_t+num_el_c+2, tn], color = 'k', linewidth=1)
    
    if len(sensor_loc)>0: # if there is any sensor
        plt.plot([sensor_loc,sensor_loc],[round(np.amin(T_var)),round(np.amax(T_var))*1.1]
                 , color = 'green', linewidth=1)
    plt.xlabel('location (m)')
    plt.ylabel('Standard Deviation')
    plt.ylim([np.amin(T_var),4])
    if len(sensor_loc)>0: # if there is any sensor
        plt.title('Sensor location (shown as green line)= {} m \nSensor is at node(s)  {}.'
                  .format(sensor_loc, sensor_loc_n), fontsize=14, loc="left")
    plt.show()




**Inputs**

- **Transitions**:
$X_t/X_{t-1} \sim f(x_t/x_{t-1})$

- **Observations**:
$Y_t/X_{t} \sim g(y_t/x_{t})$

- **Initial state** :
$X_0  \sim \mu(x_0)$

In [29]:
g_sigma = 120 # sensor measurements variance
f_sigma = 20 # transition variance
mu_sigma = 1 # initial distiribution variance
mu_mean = 0 # initial state mean
N= 10000 # number of samples
start_t = 1
end_t = 400
step_t = 1
t_out = 250 # time to draw the distribution of samples as an output
n = int(int(end_t-start_t)/step_t + 1) # number os states

**Generating synthetic data**
- True (non-observed) temperature at the location at different time. We are trying to infere this temperature

In [30]:
# Generating synthetic data
X_true = np.cumsum(np.random.normal(mu_mean,f_sigma,n)) # true (unobservable) values for x
obs = np.random.normal(X_true,g_sigma,n) # observed values

**Bootstrap filter**

In [31]:
# initialization, t=0
Sample = np.zeros((N,1))
X_0 = np.random.normal(mu_mean,mu_sigma,N) # N samples from mu ~ Normal(mu_mean,mu_sigma)
X_0 = X_0.reshape(N,1)
Sample[:,0] = np.array([X_0[:,0]]) # an array storing all the samples
X_old = X_0
for t in range(start_t,end_t+step_t,step_t):
    f_mean = X_old.reshape(N)
    X_new = np.random.normal(f_mean,f_sigma,N)
    Sample = np.append(Sample,X_new.reshape(N,1),axis=1)
    # importance sampling step 
    wnew = sp.stats.norm.pdf(obs[t-1], X_new, g_sigma) #y_pdf(g_sigma,X_new,obs[t-1])
    Wnew = wnew / sum(wnew) # normalizing the weights
    # selection step
    Sample[:,t] = np.random.choice(Sample[:,t], N, p=Wnew)
    # updating the state values
    X_old = Sample[:,t]
pred = np.sum(Sample,axis=0)/N # mean of our predictions

**Predictions:**
- Blue dots are sensors observations
- Green line shows our prediction for temperature
- Red line is the true (non-observed) temperature at the point (synthetic data)

In [32]:
interactive(lambda t=0: plot_distribution(start_t,end_t,step_t,t,X_true,obs,Sample,pred), t=(start_t,end_t,step_t*10))

interactive(children=(IntSlider(value=1, description='t', max=400, min=1, step=10), Output()), _dom_classes=('…

# Stochastic modelling of temperature in composite-tool system

If we do the bootstrap for a system with n nodes using N particles. At one node (or potentially more than one), we have the a measurment of the temperature at each time step. The measurement sensor has some uncertainty as well. The following algorithm is used here to simulate temperature.

![](Bootstrap_for_T.jpg)

**Importing the FE simulation tool** <br>
FE is used to generate data for our probabilistic modelling

In [33]:
from ipynb.fs.full.FETemp import T_air, C, alpha_dot_func, alpha_func, Mesh, Mesh3, KCF, KCF3, Assemble, Assemble3
from ipynb.fs.full.FETemp import plot_Temp, plot_T_alpha_el, plot_temp_error_shade, plot_alpha_error_shade, plot_node_temp_std
from ipynb.fs.full.FETemp import FE

**Proposed bootstrap filter for probabilistic modelling of temperature distirbution**

In [34]:
# T_true is an array with shape (number_node,number_of time steps) coming from FE solution for the entire time
# sensor_loc is a list of sensor locations with size (number_sensors)

def Temp_bootstrap(sensor_loc_typ,sensor_loc_list,obs_sigma,T_sigma,alpha_sigma,alpha_dot_sigma,
                   muT_sigma,mualpha_sigma,mualpha_dot_sigma,
                   N,t_start,t_end,delt,Length_c,Length_t,num_el_c,num_el_t,
                   Coords_start,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma,
                   material_dict,
                   Analysis,cri,Element_type,heat_gen,T_true,alpha_true,alpha_dot_true):

    Length = Length_c + Length_t
    num_el = num_el_c + num_el_t
    # material properties sampling
    # sampling
    k_c = np.random.normal(material_dict['k_c_mean'],material_dict['k_c_sigma'],N)
    rho_c = np.random.normal(material_dict['rho_c_mean'],material_dict['rho_c_sigma'],N)
    Cp_c = np.random.normal(material_dict['Cp_c_mean'],material_dict['Cp_c_sigma'],N)
    rho_r = np.random.normal(material_dict['rho_r_mean'],material_dict['rho_r_sigma'],N)
    H_r = np.random.normal(material_dict['H_r_mean'],material_dict['H_r_sigma'],N)
    nu_r = np.random.normal(material_dict['nu_r_mean'],material_dict['nu_r_sigma'],N)
    h_c = np.random.normal(material_dict['h_c_mean'],material_dict['h_c_sigma'],N)
    
    k_t = np.random.normal(material_dict['k_t_mean'],material_dict['k_t_sigma'],N)
    rho_t = np.random.normal(material_dict['rho_t_mean'],material_dict['rho_t_sigma'],N)
    Cp_t = np.random.normal(material_dict['Cp_t_mean'],material_dict['Cp_t_sigma'],N)
    h_t = np.random.normal(material_dict['h_t_mean'],material_dict['h_t_sigma'],N)
    
    # particles in FE
    A1 = np.random.normal(material_dict['A1_mean'],material_dict['A1_sigma'],N)
    A2 = np.random.normal(material_dict['A2_mean'],material_dict['A2_sigma'],N)
    A3 = np.random.normal(material_dict['A3_mean'],material_dict['A3_sigma'],N)
    dE1 = np.random.normal(material_dict['dE1_mean'],material_dict['dE1_sigma'],N)
    dE2 = np.random.normal(material_dict['dE2_mean'],material_dict['dE2_sigma'],N)
    dE3 = np.random.normal(material_dict['dE3_mean'],material_dict['dE3_sigma'],N)
    BB = np.random.normal(material_dict['BB_mean'],material_dict['BB_sigma'],N)
    a_c =  k_c/(rho_c*Cp_c)
    b_c =  rho_r*H_r*nu_r/(rho_c*Cp_c)
    Ch_c = h_c/k_c*a_c 
    a_t =  k_t/(rho_t*Cp_t)
    b_t =  np.zeros(N,)
    Ch_t = h_t/k_t*a_t
       
    
    n = int(int(t_end-t_start)/delt + 1) # number of states
    
    if sensor_loc_typ == "node":
        sensor_loc_n = sensor_loc_list # a list, node numbers
        sensor_loc = [(i-1) * (Length/num_el) for i in sensor_loc_n] 
    elif sensor_loc_typ == "loc":
        sensor_loc = sensor_loc_list # a list, location of sensor (m)
        sensor_loc_n = [int(round(x /  (Length/num_el))) + 1 for x in sensor_loc] # sensor location node number

    # Generating fake observations from T_true
    # observations is an array with shape (number_sensors,number_timestep)
    observations = np.zeros((len(sensor_loc_n),n)) # n is the number of time steps
    for sens in range(len(sensor_loc_n)): # observations if we put the sensor at i location
        observations[sens,:]  = T_true[sensor_loc_n[sens]-1,:] + np.random.normal(0,obs_sigma,n) 
        
    # initialization, t=0
    T_0_allp = np.ones((1,N)) 
    for node in range(0,num_el+1):
        muT_mean = T_true[node,0]
        T_0 = np.random.normal(muT_mean,muT_sigma,N) # N samples from mu ~ Normal(mu_mean,mu_sigma)
        T_0_allp = np.append(T_0_allp,T_0.reshape(1,N), axis=0)  
    T_old_allp = T_0_allp[1:,:]
    T_all_ave =np.mean(T_old_allp,axis=1).reshape(num_el+1,1) #np.zeros((num_el+1,1))
    T_all_var = np.zeros((num_el+1,1))
    T_all_var.fill(muT_sigma)
    
    alpha_0_allp = np.ones((1,N)) 
    alpha_dot_0_allp = np.ones((1,N)) 
    for el in range(0,num_el):
        mualpha_mean = alpha_true[el,0]
        alpha_0 = np.random.normal(mualpha_mean,mualpha_sigma,N) # N samples from mu ~ Normal(mu_mean,mu_sigma)
        alpha_0_allp = np.append(alpha_0_allp,alpha_0.reshape(1,N), axis=0) 
        mualpha_dot_mean = alpha_dot_true[el,0]
        #alpha_dot_0 = np.random.normal(mualpha_dot_mean,mualpha_dot_sigma,N) # N samples from mu ~ Normal(mu_mean,mu_sigma)
        alpha_dot_0 = halfnorm.rvs(loc = mualpha_dot_mean, scale = mualpha_dot_sigma, size = N)
        alpha_dot_0_allp = np.append(alpha_dot_0_allp,alpha_dot_0.reshape(1,N), axis=0) 
    alpha_old_allp = alpha_0_allp[1:,:]
    alpha_all_ave =np.mean(alpha_old_allp,axis=1).reshape(num_el,1)  # np.zeros((num_el,1))
    alpha_all_var =np.zeros((num_el,1))
    alpha_all_var.fill(mualpha_sigma)
    alpha_dot_old_allp = alpha_dot_0_allp[1:,:]
    alpha_dot_all_ave = np.mean(alpha_dot_old_allp,axis=1).reshape(num_el,1) # np.zeros((num_el,1))
    alpha_dot_all_var =np.zeros((num_el,1))
    alpha_dot_all_var.fill(mualpha_dot_sigma)
    
    for t in np.arange(t_start,t_end,delt):
        # Solve one step of FE for each particle to obtain new T_mean
        T_mean_allp = np.zeros((num_el+1,1))
        alpha_mean_allp = np.zeros((num_el,1))
        alpha_dot_mean_allp = np.zeros((num_el,1))
        for p in range(0,N):
            T_mean, Coords, alpha_mean, alpha_dot_mean = FE(t,t+delt,delt,Length_c,Length_t,num_el_c,num_el_t,
                                                            Coords_start,
                                                            air_temp_type,T_start,T_hold,
                                                            T_const,T_rate,th1,th2,T_air_sigma,
                                                            a_c[p],b_c[p],Ch_c[p],a_t[p],b_t[p],Ch_t[p],
                                                            BB[p],A1[p],A2[p],A3[p],dE1[p],dE2[p],dE3[p],
                                                            Analysis,cri,
                                                            Element_type,heat_gen,
                                                            T_old_allp[:,p].reshape(num_el+1,1),
                                                            alpha_old_allp[:,p].reshape(num_el,1),
                                                            alpha_dot_old_allp[:,p].reshape(num_el,1))
            
            T_mean_allp = np.append(T_mean_allp,T_mean[:,1].reshape(num_el+1,1),axis=1)
            alpha_mean_allp = np.append(alpha_mean_allp,alpha_mean[:,1].reshape(num_el,1),axis=1)
            alpha_dot_mean_allp = np.append(alpha_dot_mean_allp,alpha_dot_mean[:,1].reshape(num_el,1),axis=1)
            
        T_mean_allp = T_mean_allp[:,1:]
        alpha_mean_allp = alpha_mean_allp[:,1:]
        alpha_dot_mean_allp = alpha_dot_mean_allp[:,1:]

        # Sampling the new particles for each node/element
        T_new_allp = np.zeros((1,N))
        for node in range(0,num_el+1):
            T_new_node = np.random.normal(T_mean_allp[node,:],T_sigma,N)
            T_new_allp = np.append(T_new_allp,T_new_node.reshape(1,N), axis=0)
        alpha_new_allp = np.zeros((1,N))
        alpha_dot_new_allp = np.zeros((1,N))
        for el in range(0,num_el):
            alpha_new_el =  np.random.normal(alpha_mean_allp[el,:],alpha_sigma,N) # alpha_mean_allp[el,:]
            alpha_new_allp = np.append(alpha_new_allp,alpha_new_el.reshape(1,N), axis=0)
            #alpha_dot_new_el = np.random.halfnormal(alpha_dot_mean_allp[el,:],alpha_dot_sigma,N) # alpha_dot_mean_allp[el,:] 
            alpha_dot_new_el = halfnorm.rvs(loc = alpha_dot_mean_allp[el,:], scale = alpha_dot_sigma, size = N)
            alpha_dot_new_allp = np.append(alpha_dot_new_allp,alpha_dot_new_el.reshape(1,N), axis=0)
             
        
        # weight calculations
        Weight_allp = np.zeros((1,N))
        for sens in range(len(sensor_loc_n)): # len(sensor_loc_n) = number of srensors
            tn = int((t-t_start)/delt) # time step number
            weight = sp.stats.norm.pdf(observations[sens,tn], T_new_allp[sensor_loc_n[sens],:], obs_sigma) # sp.stats.norm.pdf(observation[node], T_new_allp[node,:], obs_sigma) 
            Weight = weight / sum(weight) # normalizing the weights
            Weight_allp = np.append(Weight_allp,Weight.reshape(1,N), axis=0)
        Weight_allp = Weight_allp[1:,:]

        # Resampling
        s = 0
        for i in range(len(sensor_loc_n)):
            T_new_allp[sensor_loc_n[i],:] = np.random.choice(T_new_allp[sensor_loc_n[i],:], N, p=Weight_allp[s,:])
            s +=1

        # updating results
        T_old_allp = T_new_allp[1:,:]
        T_old_ave = np.mean(T_old_allp,axis=1)
        T_old_var = np.var(T_old_allp, axis=1) 
        T_all_ave = np.append(T_all_ave,T_old_ave.reshape(num_el+1,1), axis=1)
        T_all_var = np.append(T_all_var,T_old_var.reshape(num_el+1,1), axis=1)
        
        alpha_old_allp = alpha_new_allp[1:,:]
        alpha_old_ave = np.mean(alpha_old_allp,axis=1)
        alpha_old_var = np.var(alpha_old_allp, axis=1) 
        alpha_all_ave = np.append(alpha_all_ave,alpha_old_ave.reshape(num_el,1), axis=1)
        alpha_all_var = np.append(alpha_all_var,alpha_old_var.reshape(num_el,1), axis=1)
        
        alpha_dot_old_allp = alpha_dot_new_allp[1:,:]
        alpha_dot_old_ave = np.mean(alpha_dot_old_allp,axis=1)
        alpha_dot_old_var = np.var(alpha_dot_old_allp, axis=1) 
        alpha_dot_all_ave = np.append(alpha_dot_all_ave,alpha_dot_old_ave.reshape(num_el,1), axis=1)
        alpha_dot_all_var = np.append(alpha_dot_all_var,alpha_dot_old_var.reshape(num_el,1), axis=1)
        
        if int((t-t_start)/delt)%5 == 0:
            clear_output(wait=True)
            print ("progress is : {}%".format(round((t-t_start)/(t_end-t_start)*100,1)))
        
    #T_all_ave = T_all_ave[:,1:]
    #T_all_var = T_all_var[:,1:]
    
    #alpha_all_ave = alpha_all_ave[:,1:]
    #alpha_all_var = alpha_all_var[:,1:]
    
    #alpha_dot_all_ave = alpha_dot_all_ave[:,1:]
    #alpha_dot_all_var = alpha_dot_all_var[:,1:]

    return T_all_ave, T_all_var, Coords, alpha_all_ave, alpha_all_var, alpha_dot_all_ave, alpha_dot_all_var,

## Examples

### Example 1 : sensor measurements at two locations

**Inputs**

In [35]:
# geometry
# composite
Length_c = 0.030 # rod length (m)
num_el_c = 10 # number of elements
# tool
Length_t = 0.015 # tool length (m)
num_el_t = 5 # number of elements
Coords_start = 0 # first node x coordinate
sensor_loc_typ1 = "node" # "node" for ndoe numbers, "loc" for locations in meter
sensor_loc_list1 = [6,13] # node numbers or location of snesors (m)

t_start = 0 # start time (seconds)
t_end = 60*60 # end time (seconds)
delt = 1 # time step (seconds)
n = int(int(t_end-t_start)/delt + 1) # number of states

In [36]:
# analysis type
Analysis = 'Forward';  # 'Backward' or 'Forward', Backward is Implicit Euler w/ Newton Raphson, Forward is Expicit Euler
cri = 0.01 # convergence criteria value for Implicit analysis
Element_type = 'Linear' # 'Linear' or 'Nonlinear'

# heat generation switch
heat_gen = 'Yes' # 'Yes' or 'No'

# air temperautre
air_temp_type = 'OneHold' # 'Constant', 'ConstantRate', 'OneHold'
T_start = 20+273 # start air temperature (K)
T_const = 180+273 # air consat temperate (for 'Constat' type) (K)
T_rate = 0.5 # air temperature increase rate (for 'Constant_rate' type)
T_hold = 170+273 # air hold temperate (for 'OneHold' type) (K)
th1 = 70*60 # time for start of hold (for 'OneHold' type) (seconds)
th2 = 170*60 # time for end of hold (for 'OneHold' type) (seconds)

In [37]:
# initial condition
num_el = num_el_c + num_el_t
T_ini = np.ones((num_el+1,1))* T_air(0,air_temp_type,T_start,
                                     T_hold,T_const,T_rate,th1,th2,T_air_sigma = 0) # initital temperature of material
alpha_ini = np.zeros((num_el,1))
alpha_dot_ini = np.zeros((num_el,1))

In [38]:
# material properties, mean values
rho_c_mean = 1463 # composites density (kg/m3) 
# --> 1463 for AS4/3501-6 composites (https://pdfs.semanticscholar.org/f069/9fb46a1958f250cc748a673e5a6b8e1910c6.pdf)
#--> 1790 for AS4 carbon (https://www.900gpa.com/en/product/fiber/CF_001EF245BC?u=metric)
k_c_mean = 0.65 # composites thermal conductivity (W/m K) 
# --> 0.65 for AS4/3501-6 composites (https://pdfs.semanticscholar.org/f069/9fb46a1958f250cc748a673e5a6b8e1910c6.pdf)
#--> 6.83 for AS4 carbon (https://www.900gpa.com/en/product/fiber/CF_001EF245BC?u=metric)
Cp_c_mean = 1200 # composite specific heat capacity (J/kg K) 
# --> 1200 for AS4/3501-6 composites (https://pdfs.semanticscholar.org/f069/9fb46a1958f250cc748a673e5a6b8e1910c6.pdf)
# --> 1300 for AS4 Carbon (https://www.researchgate.net/figure/Specific-heat-capacity-of-AS4-carbon-fiber-PES-matrix-and-CF-PES-tape-fiber-volume_fig6_320801788)
rho_r_mean = 1256 # resin density  (kg/m3), 
# -->1256 for 3501-6 (https://www.researchgate.net/figure/3-Properties-of-Hexcel-3501-6-Epoxy-Resin-17_tbl3_267585693)
H_r_mean = 400e3 # resin heat of reasction per unit mass (J / kg)
# --> 400*1000  for 3501-6 (https://books.google.ca/books?id=p__RBQAAQBAJ&pg=PA478&lpg=PA478&dq=resin+3501-6+heat+reaction+per+unit+mass&source=bl&ots=yzGE-Cu-Fo&sig=ACfU3U07FEurjhNeAVzwOKofNp-Y_zYDdw&hl=en&sa=X&ved=2ahUKEwjut6Lx2OboAhUMrp4KHf90BkAQ6AEwAHoECAsQLA#v=onepage&q=resin%203501-6%20heat%20reaction%20per%20unit%20mass&f=false)
nu_r_mean = 0.33 # resin volume fraction in composite material 
# --> 0.33
h_c_mean = 120; # convection heat trasnfer coefficient (W/ m2 K)
# --> 120 in autoclave (https://www.semanticscholar.org/paper/HEAT-TRANSFER-COEFFICIENT-DISTRIBUTION-INSIDE-AN-Slesinger-Shimizu/b61dfa6b4811edb51b003e43cc61088f0d13e348)

# tool properties
rho_t_mean = 8150; # tool density (kg/m3) 
# -->  ~ 8150 for Invar (https://www.azom.com/properties.aspx?ArticleID=515)
k_t_mean = 13; # tool thermal conductivity (W/m K) 
# --> ~13 for Invar (https://www.azom.com/properties.aspx?ArticleID=515)
Cp_t_mean = 510; # tool specific heat capacity (J/kg K) 
# --> ~ 510 for Invar (https://www.azom.com/properties.aspx?ArticleID=515)
h_t_mean = 100;

# cure kenetic
# Table 5.2 of S. Amini Niaki thesis for 3501-6
A1_mean = 3.5017e7
A2_mean = -3.3567e7
A3_mean = 3.2667e3
dE1_mean = 80700
dE2_mean = 77800
dE3_mean = 56600
# Table 5.2 of S.A. Niaki thesis for 3501-6  
BB_mean = 0.47


In [39]:
# probabilistic modelling input parameters
N = 100 # number of samples
 # uncertainty in measurements of temperature
obs_sigma = 1
# deterministc solutions uncertainties
T_sigma = 0
muT_sigma1 = 0
alpha_sigma = 0
mualpha_sigma = 0
alpha_dot_sigma = 0
mualpha_dot_sigma = 0
# uncertainties in material properties
rho_c_sigma = 3*2
k_c_sigma = 0.05*2
Cp_c_sigma = 10*2
rho_r_sigma = 6*2
H_r_sigma = 1000*2
nu_r_sigma = 0.01*2
h_c_sigma = 2*2
rho_t_sigma = 3*2
k_t_sigma = 0.05*2
Cp_t_sigma = 10*2
h_t_sigma = 2*2
A1_sigma = 0.5e7*2
A2_sigma = 0.3e7*2
A3_sigma = 0.10e3*2
dE1_sigma = 50*2
dE2_sigma = 50*2
dE3_sigma = 50*2
BB_sigma = 0.01*2
# uncertainty in air temperature
T_air_sigma1 = 0

a_c_mean =  k_c_mean/(rho_c_mean*Cp_c_mean)
b_c_mean =  rho_r_mean*H_r_mean*nu_r_mean/(rho_c_mean*Cp_c_mean)
Ch_c_mean = h_c_mean/k_c_mean*a_c_mean; 

a_t_mean =  k_t_mean/(rho_t_mean*Cp_t_mean);
b_t_mean =  0;
Ch_t_mean = h_t_mean/k_t_mean*a_t_mean; 

material_dict = {'k_c_mean':k_c_mean,'k_c_sigma':k_c_sigma,
                 'rho_c_mean':rho_c_mean,'rho_c_sigma':rho_c_sigma,
                 'Cp_c_mean':Cp_c_mean,'Cp_c_sigma':Cp_c_sigma,
                 'rho_r_mean':rho_r_mean,'rho_r_sigma':rho_r_sigma,
                 'H_r_mean':H_r_mean,'H_r_sigma':H_r_sigma,
                 'nu_r_mean':nu_r_mean,'nu_r_sigma':nu_r_sigma,
                 'h_c_mean':h_c_mean,'h_c_sigma':h_c_sigma,
                 'k_t_mean':k_t_mean,'k_t_sigma':k_t_sigma,
                 'rho_t_mean':rho_t_mean,'rho_t_sigma':rho_t_sigma,
                 'Cp_t_mean':Cp_t_mean,'Cp_t_sigma':Cp_t_sigma,
                 'h_t_mean':h_c_mean,'h_t_sigma':h_c_sigma,
                 'A1_mean':A1_mean,'A1_sigma':A1_sigma,
                 'A2_mean':A2_mean,'A2_sigma':A2_sigma,
                 'A3_mean':A3_mean,'A3_sigma':A3_sigma,
                 'dE1_mean':dE1_mean,'dE1_sigma':dE1_sigma,
                 'dE2_mean':dE2_mean,'dE2_sigma':dE2_sigma,
                 'dE3_mean':dE3_mean,'dE3_sigma':dE3_sigma,
                 'BB_mean':BB_mean,'BB_sigma':BB_sigma,}

**Generating synthetic data**

In [40]:
# true temperature
start = timeit.default_timer()

T_true, Coords, alpha_true, alpha_dot_true, = FE(t_start,t_end,delt,Length_c,Length_t,num_el_c,num_el_t,
                                                 Coords_start,air_temp_type,
                                                 T_start,T_hold,T_const,T_rate,th1,th2,0,
                                                 a_c_mean,b_c_mean,Ch_c_mean,a_t_mean,b_t_mean,Ch_t_mean,
                                                 BB_mean,A1_mean,A2_mean,A3_mean,dE1_mean,dE2_mean,dE3_mean,
                                                 Analysis,cri,Element_type,heat_gen,T_ini,alpha_ini,alpha_dot_ini)
# obervations
# fake observations are generated whiting Temp_bootstrap function. In case observation are to be given
# by user, this function should be modified accordingly. 

stop = timeit.default_timer()
print('Run time (s): ', stop - start)

Run time (s):  4.459460400000012


**Probabilistic modelling**

In [41]:
start = timeit.default_timer()

T_mean_test1, T_var_test1, Coords_test1, \
alpha_mean_test1, alpha_var_test1, alpha_dot_mean_test1, \
alpha_dot_var_test1 = Temp_bootstrap(sensor_loc_typ1,sensor_loc_list1,obs_sigma,T_sigma,alpha_sigma,alpha_dot_sigma,
                                    muT_sigma1,mualpha_sigma,mualpha_dot_sigma,
                                    N,t_start,t_end,delt,Length_c,Length_t,num_el_c,num_el_t,
                                    Coords_start,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma1,
                                    material_dict,Analysis,cri,
                                    Element_type,heat_gen,T_true,alpha_true,alpha_dot_true)

stop = timeit.default_timer()
print('Run time (s): ', stop - start)

progress is : 99.9%
Run time (s):  573.4531126


In [42]:
interactive(lambda t=0: plot_temp_error_shade(t,t_start,t_end,delt,Coords_test1,T_mean_test1,T_var_test1,air_temp_type,
                                              T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma1,T_ini,
                                              sensor_loc_typ1,sensor_loc_list1,Length_c,Length_t,num_el_c,num_el_t),
            t=(t_start,t_end,(t_end-t_start)/20))

interactive(children=(FloatSlider(value=0.0, description='t', max=3600.0, step=180.0), Output()), _dom_classes…

In [45]:
interactive(lambda node_number=1:plot_node_temp_std(node_number,T_mean_test1,T_var_test1,t_start,t_end,delt,air_temp_type,
                                                     T_start,T_hold,T_const,T_rate,th1,th2,sensor_loc_typ1,
                                                     sensor_loc_list1,Length_c,Length_t,num_el_c,num_el_t,T_air_sigma1),
            node_number=range(1,num_el+2))

interactive(children=(Dropdown(description='node_number', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, …

### Example 2:  no sensor measurements

In [46]:
sensor_loc_typ2 = "node" # "node" for ndoe numbers, "loc" for locations in meter
sensor_loc_list2 = [] # node numbers or location of snesors (m)

In [49]:
# Generating fake observation data
# true temperature
start = timeit.default_timer()

T_true, Coords, alpha_true, alpha_dot_true, = FE(t_start,t_end,delt,Length_c,Length_t,num_el_c,num_el_t,
                                                 Coords_start,air_temp_type,
                                                 T_start,T_hold,T_const,T_rate,th1,th2,0,
                                                 a_c_mean,b_c_mean,Ch_c_mean,a_t_mean,b_t_mean,Ch_t_mean,
                                                 BB_mean,A1_mean,A2_mean,A3_mean,dE1_mean,dE2_mean,dE3_mean,
                                                 Analysis,cri,Element_type,heat_gen,T_ini,alpha_ini,alpha_dot_ini)
# obervations
# fake observations are generated whiting Temp_bootstrap function. In case observation are to be given
# by user, this function should be modified accordingly. 

stop = timeit.default_timer()
print('Run time (s): ', stop - start)

Run time (s):  4.498092899999847


In [50]:
start = timeit.default_timer()

T_mean_test2, T_var_test2, Coords_test1, \
alpha_mean_test2, alpha_var_test2, alpha_dot_mean_test2, \
alpha_dot_var_test2 = Temp_bootstrap(sensor_loc_typ2,sensor_loc_list2,obs_sigma,T_sigma,alpha_sigma,alpha_dot_sigma,
                                    muT_sigma1,mualpha_sigma,mualpha_dot_sigma,
                                    N,t_start,t_end,delt,Length_c,Length_t,num_el_c,num_el_t,
                                    Coords_start,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma1,
                                    material_dict,Analysis,cri,
                                    Element_type,heat_gen,T_true,alpha_true,alpha_dot_true)

stop = timeit.default_timer()
print('Run time (s): ', stop - start)

progress is : 99.9%
Run time (s):  543.3251000999999


In [51]:
interactive(lambda t=0: plot_temp_error_shade(t,t_start,t_end,delt,Coords_test1,T_mean_test2,T_var_test2*5,air_temp_type,
                                              T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma1,T_ini,
                                              sensor_loc_typ2,sensor_loc_list2,Length_c,Length_t,num_el_c,num_el_t),
            t=(t_start,t_end,(t_end-t_start)/40))

interactive(children=(FloatSlider(value=0.0, description='t', max=3600.0, step=90.0), Output()), _dom_classes=…

###  Example 3: no sensor measurements, no uncertainty of air temperature

In [ ]:
sensor_loc_typ3 = "node" # "node" for ndoe numbers, "loc" for locations in meter
sensor_loc_list3 = [] # node numbers or location of snesors (m)

In [ ]:
# uncertainty in air temperature
T_air_sigma3 = 0
muT_sigma3 = 0

In [ ]:
# Generating fake observation data
# true temperature
start = timeit.default_timer()

T_true, Coords, alpha_true, alpha_dot_true, = FE(t_start,t_end,delt,Length_c,Length_t,num_el_c,num_el_t,
                                                 Coords_start,air_temp_type,
                                                 T_start,T_hold,T_const,T_rate,th1,th2,0,
                                                 a_c_mean,b_c_mean,Ch_c_mean,a_t_mean,b_t_mean,Ch_t_mean,
                                                 BB_mean,A1_mean,A2_mean,A3_mean,dE1_mean,dE2_mean,dE3_mean,
                                                 Analysis,cri,Element_type,heat_gen,T_ini,alpha_ini,alpha_dot_ini)
# obervations
# fake observations are generated whiting Temp_bootstrap function. In case observation are to be given
# by user, this function should be modified accordingly. 

stop = timeit.default_timer()
print('Run time (s): ', stop - start)

In [ ]:
start = timeit.default_timer()

T_mean_test3, T_var_test3, Coords_test1, \
alpha_mean_test3, alpha_var_test3, alpha_dot_mean_test3, \
alpha_dot_var_test3 = Temp_bootstrap(sensor_loc_typ3,sensor_loc_list3,obs_sigma,T_sigma,alpha_sigma,alpha_dot_sigma,
                                    muT_sigma3,mualpha_sigma,mualpha_dot_sigma,
                                    N,t_start,t_end,delt,Length_c,Length_t,num_el_c,num_el_t,
                                    Coords_start,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma3,
                                    material_dict,Analysis,cri,
                                    Element_type,heat_gen,T_true,alpha_true,alpha_dot_true)

stop = timeit.default_timer()
print('Run time (s): ', stop - start)

In [ ]:
interactive(lambda t=0: plot_temp_error_shade(t,t_start,t_end,delt,Coords_test1,T_mean_test3,T_var_test3,air_temp_type,
                                              T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma3,T_ini,
                                              sensor_loc_typ3,sensor_loc_list3,Length_c,Length_t,num_el_c,num_el_t),
            t=(t_start,t_end,(t_end-t_start)/40))

###  Example 4: no sensor, only very high uncertainty in air temperature

In [ ]:
sensor_loc_typ4 = "node" # "node" for ndoe numbers, "loc" for locations in meter
sensor_loc_list4 = [] # node numbers or location of snesors (m)

In [ ]:
obs_sigma = 1
# deterministc solutions uncertainties
T_sigma = 0
muT_sigma4 = 0
alpha_sigma = 0
mualpha_sigma = 0
alpha_dot_sigma = 0
mualpha_dot_sigma = 0
# uncertainties in material properties
rho_c_sigma = 0
k_c_sigma = 0
Cp_c_sigma = 0
rho_r_sigma = 0
H_r_sigma = 0
nu_r_sigma = 0
h_c_sigma = 0
rho_t_sigma = 0
k_t_sigma = 0
Cp_t_sigma = 0
h_t_sigma = 0
A1_sigma = 0
A2_sigma = 0
A3_sigma = 0
dE1_sigma = 0
dE2_sigma = 0
dE3_sigma = 0
BB_sigma = 0
# uncertainty in air temperature
T_air_sigma4 = 20

a_c_mean =  k_c_mean/(rho_c_mean*Cp_c_mean)
b_c_mean =  rho_r_mean*H_r_mean*nu_r_mean/(rho_c_mean*Cp_c_mean)
Ch_c_mean = h_c_mean/k_c_mean*a_c_mean; 

a_t_mean =  k_t_mean/(rho_t_mean*Cp_t_mean);
b_t_mean =  0;
Ch_t_mean = h_t_mean/k_t_mean*a_t_mean; 

material_dict = {'k_c_mean':k_c_mean,'k_c_sigma':k_c_sigma,
                 'rho_c_mean':rho_c_mean,'rho_c_sigma':rho_c_sigma,
                 'Cp_c_mean':Cp_c_mean,'Cp_c_sigma':Cp_c_sigma,
                 'rho_r_mean':rho_r_mean,'rho_r_sigma':rho_r_sigma,
                 'H_r_mean':H_r_mean,'H_r_sigma':H_r_sigma,
                 'nu_r_mean':nu_r_mean,'nu_r_sigma':nu_r_sigma,
                 'h_c_mean':h_c_mean,'h_c_sigma':h_c_sigma,
                 'k_t_mean':k_t_mean,'k_t_sigma':k_t_sigma,
                 'rho_t_mean':rho_t_mean,'rho_t_sigma':rho_t_sigma,
                 'Cp_t_mean':Cp_t_mean,'Cp_t_sigma':Cp_t_sigma,
                 'h_t_mean':h_c_mean,'h_t_sigma':h_c_sigma,
                 'A1_mean':A1_mean,'A1_sigma':A1_sigma,
                 'A2_mean':A2_mean,'A2_sigma':A2_sigma,
                 'A3_mean':A3_mean,'A3_sigma':A3_sigma,
                 'dE1_mean':dE1_mean,'dE1_sigma':dE1_sigma,
                 'dE2_mean':dE2_mean,'dE2_sigma':dE2_sigma,
                 'dE3_mean':dE3_mean,'dE3_sigma':dE3_sigma,
                 'BB_mean':BB_mean,'BB_sigma':BB_sigma,}

In [ ]:
# Generating fake observation data
# true temperature
start = timeit.default_timer()

T_true, Coords, alpha_true, alpha_dot_true, = FE(t_start,t_end,delt,Length_c,Length_t,num_el_c,num_el_t,
                                                 Coords_start,air_temp_type,
                                                 T_start,T_hold,T_const,T_rate,th1,th2,0,
                                                 a_c_mean,b_c_mean,Ch_c_mean,a_t_mean,b_t_mean,Ch_t_mean,
                                                 BB_mean,A1_mean,A2_mean,A3_mean,dE1_mean,dE2_mean,dE3_mean,
                                                 Analysis,cri,Element_type,heat_gen,T_ini,alpha_ini,alpha_dot_ini)
# obervations
# fake observations are generated whiting Temp_bootstrap function. In case observation are to be given
# by user, this function should be modified accordingly. 

stop = timeit.default_timer()
print('Run time (s): ', stop - start)

In [ ]:
start = timeit.default_timer()

T_mean_test4, T_var_test4, Coords_test1, \
alpha_mean_test4, alpha_var_test4, alpha_dot_mean_test4, \
alpha_dot_var_test4 = Temp_bootstrap(sensor_loc_typ4,sensor_loc_list4,obs_sigma,T_sigma,alpha_sigma,alpha_dot_sigma,
                                    muT_sigma4,mualpha_sigma,mualpha_dot_sigma,
                                    N,t_start,t_end,delt,Length_c,Length_t,num_el_c,num_el_t,
                                    Coords_start,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma4,
                                    material_dict,Analysis,cri,
                                    Element_type,heat_gen,T_true,alpha_true,alpha_dot_true)

stop = timeit.default_timer()
print('Run time (s): ', stop - start)

In [ ]:
interactive(lambda t=0: plot_temp_error_shade(t,t_start,t_end,delt,Coords_test1,T_mean_test4,T_var_test4,air_temp_type,
                                              T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma4,T_ini,
                                              sensor_loc_typ4,sensor_loc_list4,Length_c,Length_t,num_el_c,num_el_t),
            t=(t_start,t_end,(t_end-t_start)/40))

In [ ]:
'''interactive(lambda t=0: plot_alpha_error_shade(t,t_start,t_end,delt,Coords_test,alpha_mean_test4,alpha_var_test4,air_temp_type,
                                              T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma4,T_ini,
                                              sensor_loc_typ4,sensor_loc_list3,Length,num_el),
            t=(t_start,t_end,(t_end-t_start)/20))'''

In [ ]:
'''# rate of degree of cure (graph labels are not correct)
interactive(lambda t=0: plot_alpha_error_shade(t,t_start,t_end,delt,Coords_test,alpha_dot_mean_test4,alpha_dot_var_test4,air_temp_type,
                                              T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma4,T_ini,
                                              sensor_loc_typ4,sensor_loc_list4,Length,num_el),
            t=(t_start,t_end,(t_end-t_start)/20))'''

####  Example 5: no sensor, only very high uncertainty in air temperature, 30 elements

In [ ]:
sensor_loc_typ5 = "node" # "node" for ndoe numbers, "loc" for locations in meter
sensor_loc_list5 = [] # node numbers or location of snesors (m)

In [ ]:
num_el_c5 = 10
num_el_t5 = 5
num_el5 = num_el_c5 + num_el_t5
# initial condition
num_el = num_el_c + num_el_t
T_ini = np.ones((num_el5+1,1))* T_air(0,air_temp_type,T_start,
                                     T_hold,T_const,T_rate,th1,th2,T_air_sigma = 0) # initital temperature of material
alpha_ini = np.zeros((num_el5,1))
alpha_dot_ini = np.zeros((num_el5,1))

In [ ]:
# Generating fake observation data
# true temperature
start = timeit.default_timer()

T_true, Coords5, alpha_true, alpha_dot_true, = FE(t_start,t_end,delt,Length_c,Length_t,num_el_c5,num_el_t5,
                                                  Coords_start,air_temp_type,
                                                 T_start,T_hold,T_const,T_rate,th1,th2,0,
                                                 a_c_mean,b_c_mean,Ch_c_mean,a_t_mean,b_t_mean,Ch_t_mean,
                                                 BB_mean,A1_mean,A2_mean,A3_mean,dE1_mean,dE2_mean,dE3_mean,
                                                 Analysis,cri,Element_type,heat_gen,T_ini,alpha_ini,alpha_dot_ini)
# obervations
# fake observations are generated whiting Temp_bootstrap function. In case observation are to be given
# by user, this function should be modified accordingly. 

stop = timeit.default_timer()
print('Run time (s): ', stop - start)

In [ ]:
start = timeit.default_timer()

T_mean_test5, T_var_test5, Coords_test5, \
alpha_mean_test5, alpha_var_test5, alpha_dot_mean_test5, \
alpha_dot_var_test5 = Temp_bootstrap(sensor_loc_typ5,sensor_loc_list5,obs_sigma,T_sigma,alpha_sigma,alpha_dot_sigma,
                                    muT_sigma4,mualpha_sigma,mualpha_dot_sigma,
                                    N,t_start,t_end,delt,Length_c,Length_t,num_el_c5,num_el_t5,
                                    Coords_start,air_temp_type,T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma4,
                                    material_dict,Analysis,cri,
                                    Element_type,heat_gen,T_true,alpha_true,alpha_dot_true)

stop = timeit.default_timer()
print('Run time (s): ', stop - start)

In [ ]:
interactive(lambda t=0: plot_temp_error_shade(t,t_start,t_end,delt,Coords_test5,T_mean_test5,T_var_test5,air_temp_type,
                                              T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma4,T_ini,
                                              sensor_loc_typ5,sensor_loc_list5,Length_c,Length_t,num_el_c,num_el_t),
            t=(t_start,t_end,(t_end-t_start)/40))

####  Example 6: no sensor, only very high uncertainty in air temperature, constant temperature

In [ ]:
# heat generation switch
heat_gen6 = 'No' # 'Yes' or 'No'

# air temperautre
air_temp_type6 = 'Constant' # 'Constant', 'ConstantRate', 'OneHold'
# initial condition
num_el = num_el_c + num_el_t
T_ini6 = np.ones((num_el+1,1))* (20+273)

alpha_ini = np.zeros((num_el,1))
alpha_dot_ini = np.zeros((num_el,1))


In [ ]:
# Generating fake observation data
# true temperature
start = timeit.default_timer()

T_true, Coords, alpha_true, alpha_dot_true, = FE(t_start,t_end,delt,Length_c,Length_t,num_el_c,num_el_t,
                                                 Coords_start,air_temp_type6,
                                                 T_start,T_hold,T_const,T_rate,th1,th2,0,
                                                 a_c_mean,b_c_mean,Ch_c_mean,a_t_mean,b_t_mean,Ch_t_mean,
                                                 BB_mean,A1_mean,A2_mean,A3_mean,dE1_mean,dE2_mean,dE3_mean,
                                                 Analysis,cri,Element_type,heat_gen6,T_ini6,alpha_ini,alpha_dot_ini)
# obervations
# fake observations are generated whiting Temp_bootstrap function. In case observation are to be given
# by user, this function should be modified accordingly. 

stop = timeit.default_timer()
print('Run time (s): ', stop - start)

In [ ]:
start = timeit.default_timer()

T_mean_test6, T_var_test6, Coords_test1, \
alpha_mean_test6, alpha_var_test6, alpha_dot_mean_test6, \
alpha_dot_var_test6 = Temp_bootstrap(sensor_loc_typ4,sensor_loc_list4,obs_sigma,T_sigma,alpha_sigma,alpha_dot_sigma,
                                    muT_sigma4,mualpha_sigma,mualpha_dot_sigma,
                                    N,t_start,t_end,delt,Length_c,Length_t,num_el_c,num_el_t,
                                    Coords_start,air_temp_type6,T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma4,
                                    material_dict,Analysis,cri,
                                    Element_type,heat_gen6,T_true,alpha_true,alpha_dot_true)

stop = timeit.default_timer()
print('Run time (s): ', stop - start)

In [ ]:
interactive(lambda t=0: plot_temp_error_shade(t,t_start,t_end,delt,Coords_test1,T_mean_test6,T_var_test6,air_temp_type6,
                                              T_start,T_hold,T_const,T_rate,th1,th2,T_air_sigma4,T_ini6,
                                              sensor_loc_typ4,sensor_loc_list4,Length_c,Length_t,num_el_c,num_el_t),
            t=(t_start,t_end,(t_end-t_start)/40))